In [ ]:
# TEST_USCFCsv2OADCloud

## USCFCsvPUREcxORACLE

Carga del dataset desde local hacia Oracle Cloud utilizando pure cx_ORACLE.
Iteramos en batchs definidos de 100K rows, tomando en cuenta la limitacion de los buffers del array a 2GB.
DATA_IN_PATH determina el path al fichero con el Dataset. Ver presentacion para detalle de la URL para bajarlo.

In [ ]:
import sys
import cx_Oracle
import getpass
from datetime import datetime
import csv
import pandas as pd
import time

# TODO Agregar soporte para modulo logging

# -- Datetime format
STRFMT = "%m/%d/%Y, %H:%M:%S.%f"

# -- Data Path con el dataset a cargar 
DATA_IN_PATH=r"C:\data\33_793516_bundle_archive\consumer_complaints.csv"

# -- Definicion del PATH donde hemos instalado Oracle Instant Client
# -- Inicializaremmos de forma explicita
ORA_INST_CLIENT_PATH = r"C:\instantclient_19_8"
# -- 
ORA_INST_CLIENT_TNSC = r"<TNS_ENTRADA>"

# -- Dependiente de la limitacion maxima 1GB para el total del array
BATCH_ARRAY_SIZE = 100_000

# -- Solicitamos usuario y password, no recomiendo hardcode de los
# -- mismos en el codigo.
oc_user = input('AD user')
oc_pass = getpass.getpass('AD password')

# -- Ping DB Connection
# -- c = cx_Oracle connect object
# -- return True Si ping OK o False si no
# TODO (Sebas) Implmentar retries a futuro
def isConnOpen(c):
    try:
        return c.ping() is None
    except:
        return False

# -- Chunking df, existe el verbo XD?
# -- idf = Pandas DataFrame
# -- return None
# TODO (Sebas) Hacer los cortes en funcion del size de entrada
def chunkdf(idf, s=BATCH_ARRAY_SIZE):
    return( idf[i:i+s] for i in range(0, len(idf),s) )

# -- A ver si Yoda tenia razon.
dbx = None 
dbc = None

df = None
data = None

df = pd.read_csv(DATA_IN_PATH)

print('{0} [INFO] Uso de memoria {1}'.format(datetime.now().strftime(STRFMT),\
    df.memory_usage().sum()))

nrows = df.shape[0]

    # -- Previsualizamos metadatos
print(df.info())

    # -- Nos conectamos
dbx = cx_Oracle.connect(oc_user, oc_pass, ORA_INST_CLIENT_TNSC)

    # -- Autocommit a True
dbx.autocommit = True

    # -- Abrir un cursor
dbc = dbx.cursor()

# -- Preprocesar nan y nat
data = df.where(df.notnull(), None).values.tolist()
   
print('{0} [INFO] Inicio de la carga en base de datos'.format(datetime.now().strftime(STRFMT)))
start_t = time.time() 

i = 0

for idf in chunkdf(df):
    i += 1
    # -- 496 MB total en BBDD comprimidos con QUERY HIGH
    data = idf.where(idf.notnull(), None).values.tolist()
    dbc.executemany("""INSERT INTO TEST_USCF02 
               VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10
               , :11, :12, :13, :14, :15, :16, :17, :18)""", data, batcherrors=False)

    print('{0} [INFO] Finalizada la iteracion {1}'.format(datetime.now().strftime(STRFMT), i))
end_t = time.time()
print('{0} [INFO] Carga de {1} filas en tiempo {2}'.format(datetime.now().strftime(STRFMT), \
        nrows, end_t-start_t))

dbc.close()
dbx.close()
    
print('{0} [INFO] Fin del Test'.format(datetime.now().strftime(STRFMT)))
